In [1]:

%pip install trimesh scipy pygltflib

   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
    --------------------------------------- 0.8/41.0 MB 5.3 MB/s eta 0:00:08
   - -------------------------------------- 1.6/41.0 MB 4.6 MB/s eta 0:00:09
   -- ------------------------------------- 2.4/41.0 MB 4.7 MB/s eta 0:00:09
   -- ------------------------------------- 2.9/41.0 MB 3.9 MB/s eta 0:00:10
   --- ------------------------------------ 3.4/41.0 MB 3.4 MB/s eta 0:00:12
   --- ------------------------------------ 3.7/41.0 MB 3.1 MB/s eta 0:00:12
   ---- ----------------------------------- 4.2/41.0 MB 3.0 MB/s eta 0:00:13
   ---- ----------------------------------- 5.0/41.0 MB 3.2 MB/s eta 0:00:12
   ----- ---------------------------------- 6.0/41.0 MB 3.3 MB/s eta 0:00:11
   ------ --------------------------------- 6.6/41.0 MB 3.3 MB/s eta 0:00:11
   ------ --------------------------------- 6.8/41.0 MB 3.2 MB/s eta 0:00:11
   ------- -------------------------------- 7.6/41.0 MB 3.2 MB/s eta 0:00:11
   ---


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import trimesh
import numpy as np
import os

def analizar_modelo(modelo, nombre):
    """ Realiza un análisis básico de un modelo 3D """
    print(f"🔍 Análisis del modelo {nombre}:")

    # Si es una escena (GLB con varias mallas), combinar todas
    if isinstance(modelo, trimesh.Scene):
        if not modelo.geometry:
            print("- ⚠️ Escena vacía")
            return
        modelo = trimesh.util.concatenate([g for g in modelo.geometry.values()])

    # Propiedades básicas
    print(f"- Vértices: {len(modelo.vertices)}")
    print(f"- Caras: {len(modelo.faces)}")

    # Normales
    try:
        _ = modelo.vertex_normals
        print(f"- Normales presentes: Sí")
    except Exception as e:
        print(f"- Normales presentes: No ({str(e)})")

    # Watertight (cerrado)
    print(f"- ¿Está cerrado (watertight)?: {'Sí' if modelo.is_watertight else 'No'}")

    # Vértices duplicados
    unicos = np.unique(modelo.vertices, axis=0)
    duplicados = len(modelo.vertices) - len(unicos)
    if duplicados:
        print(f"- Vértices duplicados: {duplicados} ({duplicados / len(modelo.vertices) * 100:.2f}%)")
    else:
        print("- No hay vértices duplicados")

    # Bounding box y volumen (si aplica)
    print(f"- Bounding box (dimensiones): {modelo.bounding_box.extents}")
    if modelo.is_volume:
        print(f"- Volumen estimado: {modelo.volume:.3f} unidades³")

    print()
    return modelo  # Retorna el modelo para posteriores conversiones


def convertir_formato(modelo, ruta_salida):
    """ Convierte el modelo a diferentes formatos y lo guarda en una ruta específica """
    try:
        # Detecta la extensión del archivo de salida y exporta al formato adecuado
        formato = os.path.splitext(ruta_salida)[1][1:].upper()
        modelo.export(ruta_salida)
        print(f"\n✅ Conversión exitosa a {formato} en: {ruta_salida}")
    except Exception as e:
        print(f"\n❌ Error en conversión: {str(e)}")


def comparar_modelos(modelos, nombres):
    """ Compara los modelos entre sí y muestra los resultados en formato tabular """
    print("\n📊 Comparacion entre modelos:")

    # Encabezado de la tabla
    header = f"{'Modelo':<15} | {'Vertices':<10} | {'Caras':<10} | {'Duplicados':<12} | {'Cerrado'}"
    separator = '-' * len(header)
    
    print(header)
    print(separator)
    
    # Comparación de los modelos
    for modelo, nombre in zip(modelos, nombres):
        if isinstance(modelo, trimesh.Scene):
            modelo = trimesh.util.concatenate([g for g in modelo.geometry.values()])
        
        vertices = len(modelo.vertices)
        caras = len(modelo.faces)
        duplicados = len(modelo.vertices) - len(np.unique(modelo.vertices, axis=0))
        cerrado = 'Si' if modelo.is_watertight else 'No'
        
        # Mostrar fila en la tabla
        print(f"{nombre:<15} | {vertices:<10} | {caras:<10} | {duplicados:<12} | {cerrado}")
    
    print(separator)
    
    # Guardar la comparación en un archivo de texto
    with open("../resultados/comparacion_modelos.txt", "w") as file:
        file.write(header + '\n')
        file.write(separator + '\n')
        
        for modelo, nombre in zip(modelos, nombres):
            if isinstance(modelo, trimesh.Scene):
                modelo = trimesh.util.concatenate([g for g in modelo.geometry.values()])
            
            vertices = len(modelo.vertices)
            caras = len(modelo.faces)
            duplicados = len(modelo.vertices) - len(np.unique(modelo.vertices, axis=0))
            cerrado = 'Sí' if modelo.is_watertight else 'No'
            
            file.write(f"{nombre:<15} | {vertices:<10} | {caras:<10} | {duplicados:<12} | {cerrado}\n")
    
    print("\n📝 La comparación ha sido guardada en 'comparacion_modelos.txt'.\n")


def cargar_modelos(directorio):
    """ Carga todos los modelos en un directorio especificado """
    modelos = []
    nombres = []
    
    # Cargar todos los archivos con extensión .obj, .stl, .glb, etc.
    for archivo in os.listdir(directorio):
        ruta = os.path.join(directorio, archivo)
        if archivo.endswith(('.obj', '.stl', '.glb', '.gltf')):
            try:
                modelo = trimesh.load(ruta)
                modelos.append(modelo)
                nombres.append(archivo)
                print(f"📥 Modelo cargado: {archivo}")
            except Exception as e:
                print(f"❌ No se pudo cargar el archivo {archivo}: {str(e)}")
    
    return modelos, nombres


# ---------- PROGRAMA PRINCIPAL ----------

# Cargar los modelos desde el directorio ../material/
directorio_modelos = '../material/'
modelos, nombres = cargar_modelos(directorio_modelos)

# Analizar cada modelo
modelos_analizados = [analizar_modelo(modelo, nombre) for modelo, nombre in zip(modelos, nombres)]

# Comparar los modelos cargados
comparar_modelos(modelos_analizados, nombres)

# Opcional: Convertir los modelos a tres formatos (OBJ, STL, GLB)
for modelo_analizado, nombre in zip(modelos_analizados, nombres):
    base_nombre = os.path.splitext(nombre)[0]
    
    # Convertir a OBJ, STL, y GLB
    convertir_formato(modelo_analizado, f"../resultados/converted_{base_nombre}.obj")
    convertir_formato(modelo_analizado, f"../resultados/converted_{base_nombre}.stl")
    convertir_formato(modelo_analizado, f"../resultados/converted_{base_nombre}.glb")

unable to load materials from: FinalBaseMesh.mtl


❌ No se pudo cargar el archivo Cap.stl: `ptp` was removed from the ndarray class in NumPy 2.0. Use np.ptp(arr, ...) instead.
❌ No se pudo cargar el archivo Cuerpo.obj: module 'numpy' has no attribute 'product'
❌ No se pudo cargar el archivo IronMan.gltf: [Errno 2] No such file or directory: 'c:\\Users\\Justi\\OneDrive\\Documentos\\GitHub\\Computacion_Visual\\2025-05-04_taller_conversion_formatos_3d\\material\\result.bin'

📊 Comparacion entre modelos:
Modelo          | Vertices   | Caras      | Duplicados   | Cerrado
------------------------------------------------------------------
------------------------------------------------------------------

📝 La comparación ha sido guardada en 'comparacion_modelos.txt'.



In [ ]:
%pip install vedo


In [ ]:
#Visualizacion de modelos y sus propiedades

import plotly.graph_objects as go
from IPython.display import Markdown, display

def propiedades_mesh(mesh):
    """Calcula propiedades clave de un trimesh.Trimesh."""
    props = {}
    props['Vértices']    = len(mesh.vertices)
    props['Caras']        = len(mesh.faces)
    props['Cerrado']      = mesh.is_watertight
    # Volumen sólo si es volumétrico
    props['Volumen']      = mesh.volume if mesh.is_volume else None
    # Dimensiones del bounding box
    props['BBox extents']= tuple(mesh.bounding_box.extents)
    return props

def visualizar_con_propiedades(modelos, nombres, width=640, height=480):
    """Muestra cada modelo 3D en Jupyter con Plotly y sus propiedades."""
    for modelo, nombre in zip(modelos, nombres):
        # Aplanar escenas a un solo mesh
        mesh = modelo
        if isinstance(modelo, trimesh.Scene):
            mesh = trimesh.util.concatenate([g for g in modelo.geometry.values()])
        
        # --- 1) Render Plotly ---
        v = mesh.vertices
        f = mesh.faces
        fig = go.Figure(data=[
            go.Mesh3d(
                x=v[:,0], y=v[:,1], z=v[:,2],
                i=f[:,0], j=f[:,1], k=f[:,2],
                opacity=0.5,
                name=nombre
            )
        ])
        fig.update_layout(
            title=f"Vista 3D de {nombre}",
            width=width, height=height,
            scene=dict(aspectmode='data')
        )
        fig.show()
        
        # --- 2) Cálculo de propiedades ---
        props = propiedades_mesh(mesh)
        
        # --- 3) Impresión en Markdown ---
        md = f"**Propiedades de `{nombre}`**  \n"
        md += "| Propiedad      | Valor |\n"
        md += "|--------------- |:-----:|\n"
        md += f"| Vértices       | {props['Vértices']} |\n"
        md += f"| Caras           | {props['Caras']} |\n"
        md += f"| Cerrado         | {'Sí' if props['Cerrado'] else 'No'} |\n"
        if props['Volumen'] is not None:
            md += f"| Volumen         | {props['Volumen']:.3f} |\n"
        else:
            md += f"| Volumen         | N/A |\n"
        bbox = props['BBox extents']
        md += f"| Bounding box    | {bbox} |\n"
        
        display(Markdown(md))
        print()  # Espacio entre modelos

# Llamada al bloque
visualizar_con_propiedades(modelos_analizados, nombres)

